In [1]:

# TODO:
# agentificar el script y divivir en 5 notebooks
# 1. llm1 ollama finanucial
# 2. llm2 hugginface financial
# 3. web 1 trading view
# 4. web 2 investing, prorealtime, metatrader ninja trader
# 5. manual calculation with financial indicators based on historical data


# el script tiene que ejecutarse a diario

# reporte de indices bursatiles interesantes tipo NASDAQ100

# escribir un CSV que se irá engordando con los simbolos interesantes a comprar despues del analisis
# contendrá: 
# simbolo, 
# precio de compra, 
# fecha teorica de compra (datetime.now()),
# valor objetivo, 
# fecha en la que se alcanzo el valor objetivo, 
# cuantos dias pasaron entre la compra y el alcance del valor objetivo
# el CSV se adjuntará en el reporte diaro via email para ir viendo la trazabilidad y que realmente el analisis esta bien hecho y hay beneficio

In [2]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [3]:
import warnings

# Suprimir todos los warnings
warnings.filterwarnings("ignore")

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
aplha_vantage_api_key = os.getenv('ALPHA_VANTAGE_API_KEY')
history_days=365

#print("openai_api_key: "+openai_api_key+"\n"+"aplha_vantage_api_key:"+aplha_vantage_api_key)

In [4]:
# COGER EL HISTORICO
import requests
from bs4 import BeautifulSoup
import re

import requests
import pandas as pd
from datetime import datetime, timedelta

import requests
from datetime import datetime, timedelta

def get_symbol_history_from_alpha(symbol, api_key, api_days):

    # URL base de la API
    base_url = 'https://www.alphavantage.co/query'
    
    # Parámetros para la API
    params = {
        'function': 'TIME_SERIES_DAILY',
        'symbol': symbol,
        'apikey': api_key,
        'outputsize': 'full',  # Esto asegura que se obtiene todo el histórico disponible
    }
    
    # Realizar la solicitud a la API
    response = requests.get(base_url, params=params,verify=False)
    
    # Verificar si la respuesta fue exitosa
    if response.status_code == 200:
        data = response.json()
        if 'Time Series (Daily)' in data:
            # Extraer el histórico de datos
            time_series = data['Time Series (Daily)']
    
            # Definir una lista para almacenar los datos filtrados
            historical_data = []
    
            # Filtrar el histórico de los últimos 365 días
            one_year_ago = datetime.now() - timedelta(days=api_days)
    
            for date, daily_data in time_series.items():
                # Convertir la fecha a un objeto datetime
                date_obj = datetime.strptime(date, '%Y-%m-%d')
    
                # Solo agregar los datos si la fecha es dentro del último año
                if date_obj >= one_year_ago:
                    historical_data.append({
                        'date': date_obj,
                        'open': daily_data['1. open'],
                        'high': daily_data['2. high'],
                        'low': daily_data['3. low'],
                        'close': daily_data['4. close'],
                        'volume': daily_data['5. volume'],
                    })
    
            # Mostrar los resultados
            for entry in historical_data:
                print(f"Fecha: {entry['date'].strftime('%Y-%m-%d')}, "
                      f"Apertura: {entry['open']}, "
                      f"Máximo: {entry['high']}, "
                      f"Mínimo: {entry['low']}, "
                      f"Cierre: {entry['close']}, "
                      f"Volumen: {entry['volume']}")
        else:
            print("No se encontraron datos para el símbolo especificado.")
    else:
        print("Error al obtener los datos:", response.status_code)


In [5]:
# CALCULOS PARA EVALUAR SI UNA ACCION ES INTERESANTE 

import pandas as pd
import numpy as np

def calcular_sma(datos, window):
    return datos['Close'].rolling(window=window).mean()

def calcular_rsi(datos, window):
    delta = datos['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calcular_bollinger_bands(datos, window):
    sma = calcular_sma(datos, window)
    std = datos['Close'].rolling(window=window).std()
    upper_band = sma + (std * 2)
    lower_band = sma - (std * 2)
    return upper_band, lower_band

def es_interesante_comprar(datos):
    # Asegurarse de que los datos estén ordenados por fecha
    datos = datos.sort_index()

    # Calcular indicadores
    sma_50 = calcular_sma(datos, 50)
    rsi_14 = calcular_rsi(datos, 14)
    upper_band, lower_band = calcular_bollinger_bands(datos, 20)

    # Obtener valores actuales
    precio_cierre_actual = datos['Close'].iloc[-1]
    sma_actual = sma_50.iloc[-1]
    rsi_actual = rsi_14.iloc[-1]
    upper_band_actual = upper_band.iloc[-1]
    lower_band_actual = lower_band.iloc[-1]

    # Decidir si es interesante comprar
    # Condiciones:
    # 1. Precio de cierre actual > SMA de 50 días (indicador de tendencia alcista)
    # 2. RSI < 30 (indicador de sobreventa)
    # 3. Precio de cierre actual cerca de la banda inferior de Bollinger (indicador de posible reversión)
    if (precio_cierre_actual > sma_actual) and (rsi_actual < 30) and (precio_cierre_actual < lower_band_actual):
        return True
    else:
        return False

# Ejemplo de uso
datos = pd.DataFrame({
    'Open': [144.533990, 151.387397, 150.401486, 139.078055, 141.183893],
    'High': [150.994947, 154.134496, 150.947071, 146.898202, 154.976845],
    'Low': [143.576811, 146.046338, 144.677548, 132.645813, 134.703788],
    'Close': [144.170258, 154.029205, 147.051361, 133.105255, 152.028732],
}, index=pd.to_datetime([
    '2020-03-09 00:00:00-04:00',
    '2020-03-10 00:00:00-04:00',
    '2020-03-11 00:00:00-04:00',
    '2020-03-12 00:00:00-04:00',
    '2020-03-13 00:00:00-04:00'
]))

print(es_interesante_comprar(datos))

False


In [6]:
# EVALUACION DE INTERES
openai = OpenAI()
import httpx
openai = OpenAI(http_client=httpx.Client(verify=False))
llm_temperature = 0.1

def get_analysis(symbols_dictionary):
    #print("analyzing interesting symbols...")
    messages_prompt = [
            {"role": "system", "content": "you are a financial assistant"},
            {"role": "user", "content": (
                                        f"please return me a clear answer about the provided symbol and its historical data {symbols_dictionary}. "
                                        f"take into account that my intention in case of buying is to get benefit inside the following month with a 10/12% of profit"                        
                                        f"the answer has to be: 'sell', 'hold' or 'buy' for TODAY or 'no dada' (in case there is not historical data),"
                                        f"and a brief explanation about the reason, JUST IN A FEW WORDS (max. 15)."                                        
                                        )
            }
        ]
    
    response = openai.chat.completions.create(
        model = "gpt-4",
        messages = messages_prompt,
        temperature=llm_temperature
    )
    return response.choices[0].message.content

In [7]:
import requests
from bs4 import BeautifulSoup
import re

def get_market_losers():
    # URL de la página de Google Finance
    url = "https://www.google.com/finance/markets/losers?hl=es"
    
    # Realizar la solicitud HTTP a la página
    response = requests.get(url, verify = False)
    
    # Analizar el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    ul_element = soup.find_all('li')
    
    symbols_list = []
    pattern = r"^[A-Z]+"
    # Iterar sobre todos los elementos <li> dentro del <ul>
    for li in ul_element:
        symbols_list.append(re.match(pattern, li.text).group()[:-1])
    
    return symbols_list

In [8]:
# atacar a cada una de estos links 
import yfinance as yf
import requests

def get_llm_opinion(symbol):
    print(f"getting LLM opinion about {symbol}")
    session = requests.Session()
    session.verify = False
    symbols_dictionary = {}
    
    try:
        #print(f"trying to get historical data from symbol: {symbol}")
        data = yf.Ticker(symbol, session=session)
        historical_data = data.history(period="5Y")
        if not historical_data.empty:
            symbols_dictionary[symbol] = historical_data
        else:
            print(f"no data on yahoo finance for {symbol}, trying to get historical data on alpha vantage")
            historical_data = get_symbol_history_from_alpha(symbol, aplha_vantage_api_key, history_days)

            symbols_dictionary[symbol] = historical_data
            
        #print(symbols_dictionary[symbol])
        
    except Exception as e:
        print(f"Ocurrió un error: {e}")
    
    if symbols_dictionary:
        llm_answer = get_analysis(symbols_dictionary)
        return llm_answer

#get_llm_opinion('MSFT')

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re

def get_html(urls,symbol):
    try: 
        # Configurar el servicio de ChromeDriver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service)
        driver.implicitly_wait(10)  # Esperar a que la página se cargue completamente
    
    
        for url in urls:
            try:
                driver.get(url)
                # Verificar si existe un elemento con la clase 'tv-http-error-page__image'
                element = driver.find_element(By.CSS_SELECTOR, 'div[class*="speedometersContainer"]')
                #print(f"Succesfully loaded {url}")
                return driver.page_source
            except Exception as e:
                a=1
                #print(f'Error opening: {url}')
              
    except Exception as e:
        print(f"Error inesperado: {e}")
        return None
    finally:
        driver.quit()
        


In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re

def get_trading_view_opinion(symbol):
    print(f"getting trading view opinion about {symbol}...")
    try: 

        urls = [
            f'https://en.tradingview.com/symbols/BME-{symbol}/technicals/',
            f'https://en.tradingview.com/symbols/MARKETSCOM-{symbol}/technicals/',
            f'https://en.tradingview.com/symbols/NASDAQ-{symbol}/technicals/',
            f'https://en.tradingview.com/symbols/NYSE-{symbol}/technicals/'
        
        ]
        
        html = get_html(urls,symbol)
        
        if html is None:
            print('HTML is None')
            return None
        
        # Crear el objeto BeautifulSoup
        soup = BeautifulSoup(html, 'lxml')
        #print("gathering elements...")
        # Encontrar todos los elementos <span> cuya clase contenga "counterNumber-"
        span_elements = soup.find_all('div', class_=lambda x: x and 'speedometerWrapper' in x)
        
        # Acceder al contenido de los elementos <span>
        if len(span_elements) == 0:
            print("list is empty")
        else:
        
            for span in span_elements:
                if 'Summary' in span.text:
                    speedometer = span.text
                    # Usar una expresión regular para encontrar todas las palabras seguidas de números
                    
                    matches = re.findall(r'(Sell|Neutral|Buy)(\d+)', speedometer)
                    
                    # Convertir los resultados a una lista de tuplas (palabra, número)
                    resultados = [(palabra, int(numero)) for palabra, numero in matches]
                    
                    # Encontrar la tupla con el número más alto
                    max_resultado = max(resultados, key=lambda x: x[1])
                    
                    # Separar la palabra y el número con el valor más alto
                    palabra_maxima = max_resultado[0].upper()
                    numero_maximo = max_resultado[1]
                    
                    # Formatear el resultado principal
                    resultado_principal = f"{palabra_maxima} ({numero_maximo})"
                    
                    # Eliminar el resultado principal de la lista de resultados
                    resultados.remove(max_resultado)
                    
                    # Formatear el resto de los resultados
                    resultados_formateados = [f"{palabra} ({numero})" for palabra, numero in resultados]
                    
                    # Combinar el resultado principal con el resto
                    trading_view_opinion = f"{resultado_principal} - " + " - ".join(resultados_formateados)
                    return trading_view_opinion                    
    
        
    except Exception as e:
        # Manejar cualquier otra excepción
        print(f"Error inesperado: {e}")

In [11]:
#print answers

symbols = get_market_losers()
#symbols = ['MSFT','PUIG','KO','TSLA']
symbols = ['MSFT']

print("market losers: "+str(symbols))

opinions = []

for symbol in symbols:    
    answer_trading_view = get_trading_view_opinion(symbol)
    opinion_tv = {
                "symbol": symbol,
                "source": "Trading view.com",
                "opinion": answer_trading_view               
            }
    llm_answer = get_llm_opinion(symbol)
    opinion_llm = {
                "symbol": symbol,
                "source": "LLM",
                "opinion": llm_answer               
            }
    opinions.append(opinion_tv)
    opinions.append(opinion_llm)
    
    #get_investing_opinion()

market losers: ['MSFT']
getting trading view opinion about MSFT...
getting LLM opinion about MSFT


In [12]:
from IPython.display import display, Markdown


# Construir la cadena de Markdown
markdown_content = "# Trading Advisor Daily Report\n\n"

markdown_content += "| Symbol | Source | Opinion |\n"
markdown_content += "|--------|--------|---------|\n"

for opinion in opinions:
     markdown_content += f"| {opinion['symbol']} | {opinion['source']} | {opinion['opinion']}|\n"

# Mostrar la tabla de Markdown
display(Markdown(markdown_content))

# Trading Advisor Report

| Symbol | Source | Opinion |
|--------|--------|---------|
| MSFT | Trading View | SELL (15) - Neutral (10) - Buy (1)|
| MSFT | LLM | 'Buy', due to consistent growth trend over the years.|
